<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL <mark>2023</mark>.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 3.1: Morphology</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: <mark>Thursday December 14, 2023</mark></p>
    <p style="margin:4px;"><b>Submission</b>: <mark><span style="color:red">Monday December 25, 2023</span></mark> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weight</b>: Lab 3 (<mark>18</mark> points), 10% of the overall grade</p> 
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 4.3</p> 
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports

In [ ]:
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import skimage

from interactive_kit import imviewer as viewer

# Load images to be used in this lab
from skimage import io
test_contact = io.imread('images/test-contact.tif')
test_scratch = io.imread('images/test-scratch.tif')
test_img = io.imread('images/test-skeleton.tif')
hela = io.imread('images/hela.tif')
christmas = io.imread('images/christmas-tree.tif')
hela = io.imread('images/hela.tif')

Now run the next cell to re-declare the functions that create different types of structuring elements from the first part of this lab. 

In [ ]:
def square(n):
    return np.ones((n, n)).astype(np.uint8)

def disc(n):
    # Define the function of a circle as a lambda function
    circle_func = lambda i, j: ((i - n//2)**2 + (j - n//2)**2) <= (n//2)**2
    # Set all elements of the array that are inside the circle of diamater n to 1
    output = np.fromfunction(circle_func, shape=(n,n)).astype(np.uint8)
    # Return the structuring element
    return output

def cross(n):
    output = np.zeros((n, n))
    output[n//2, :] = 1
    output[:, n//2] = 1
    output = output.astype(np.uint8)
    return output

def diag(n):
    return np.flip(np.diag(np.ones(n, dtype='uint8')), axis=0)

# Morphology Applications (5 points)
In the previous part of the lab, you looked in detail at the pixel-wise implementation of morphological operations. In this second part, we will look at the applications of these operations, and how they are used in image processing workflows using skimage. **This notebook is purely in Python**.

## <a id="ToC_2_Morphology"></a>Table of contents

1. [Direct applications](#1.-Direct-applications-(3-points)) (**3 points**)
    1. [Disconnecting round objects](#1.A.-Disconnecting-round-objects)
    2. [Detecting horizontal lines](#1.B.-Detecting-horizontal-lines)
    3. [Combining morphological filters](#1.C.-Combining-morphological-filters)
2. [Lantuéjoul's skeleton](#2.-Lantuéjoul's-skeleton-(1-point)) (**1 point**)
    1. [Implementing skeletonize](#2.A.-Implementing-skeletonize)
    2. [Testing skeletonize](#2.B.-Testing-skeletonize)
3. [Distance from border](#3.-Distance-from-border-(1-point)) (**1 point**)
4. [Cartoonize your picture!](#4.-Cartoonize-your-picture!)

# 1. Direct applications (2 points)
[Back to table of contents](#ToC_2_Morphology)

For the tasks in this section 1.A and 1.B, choose **only one** of the morphological operators presented in [the first part of the lab](./1_Morphology_Implementations.ipynb#3.-Morphological-filters-(9-points)), with a suitable structuring elment of a suitable size.
If the resulting image is not binary, apply a threshold operation (e.g. using [`np.where()`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)) with an appropriate threshold value to get a binary image of values $0$ or $255$ (use `image.astype(uint8)` to convert the data type).


## 1.A. Disconnecting round objects
[Back to table of contents](#ToC_2_Morphology)

In the image `test_contact`, **for 1 point**, disconnect the roundish white objects while keeping at least 1 white pixel per roundish object.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="test-contact showcase" src="images/test-contact_showcase.png" width="500"><br>
  <em style="color: grey">Using only one morphological operator, you can separate all the roundish objects, as shown above.</em>
  </p>
</td>
</tr></table>

Run the next cell to visualize the original image. Then insert your code into the function `disconnect`, to specify the morphological operation and the structuring element to use.

<div class="alert alert-info">
<b>Hints</b>: <ul><li>In this exercise you don't need to explicitly threshold anything because the original image is already binary.</li><li>Remember to try the different parameters of a morphological operation! For example, you can change the operator, the size and the type of a structural element.</li></ul>
</div>

In [ ]:
plt.close('all')
img_vis = viewer(test_contact)

In [ ]:
# Function that performs a single morphological operation with some structuring element of size n
def disconnect(img, n):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE

    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [ ]:
# Check that the output is binary
check_bin = disconnect(test_contact, 25)
if not np.all(np.unique(check_bin)==[0,255]):
    print('WARNING!\nThe output is not binary with values {0, 255}.')
else:
    print("Good, the output is binary with values {0,255}.")

To make it easy for you to find the right size of the structuring element, we will add an interactive slider to the image display using the `ImageViewer` class. Run the next cell to use the interactive widget.
<div class="alert alert-info">

<b>Note:</b> To use the slider, click the the button <code>Extra Widgets</code>. Then you can adjust the size of the structuring element with the slider and click the button <code>Disconnect</code> to apply the <code>disconnect()</code> function with the currently selected size on the original image.
</div>

In [ ]:
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the diconnect button
button = widgets.Button(description = 'Disconnect')

# Define the callback function of the button
def button_callback(image):
    # run the disconnect function on the image with the size indicated by the slider
    output = disconnect(image, n=size_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_contact_display = viewer(test_contact, title="Disconnect test_contact", new_widgets=[size_slider, button], callbacks=[button_callback], widgets=True)

In the following cell, assign `n` with the size of the structuring element you think works best for this task. This number may be different depending on the structuring element you used in the `disconnect()` function.

In [ ]:
# Assign the size of the structuring element
n = None
# YOUR CODE HERE

In [ ]:
# Perform a sanity check on n
if not 0 < n < 50: 
    print('WARNING!\nThe chosen size doesn\'t really make sense.')

## 1.B. Detecting horizontal lines
[Back to table of contents](#ToC_2_Morphology)

In the image `test_scratch`, **for 1 point**, detect the horizontal white lines of thickness of 1 or more pixels.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="test-scratch showcase" src="images/test-scratch_showcase.png" width="500"><br>
<em style="color: grey">With <b>only one</b> morphological operation and a binarization, you can extract the second image from the first one.</em>
  </p>
</td>
</tr></table>

Run the next cell to visualize the original image, then insert your code into the function `detect_hlines(img, n, threshold)` to specify the morphological operator, the structuring element and the binary threshold to use.

In [ ]:
plt.close('all')
img_vis = viewer(test_scratch)

In [ ]:
# Function that performs a single morphological filter with some structuring element of size n followed by thresholding 
def detect_hlines(img, n, threshold):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE

    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [ ]:
# You can enter any values that work for you for the size and threshold
n = 3
threshold = 125

# Check that the output is binary
check_bin = detect_hlines(test_scratch, n, threshold)
if not (len(np.unique(check_bin)) == 2 and np.allclose(np.unique(check_bin), [0, 255])):
    print('WARNING!\nThe output is not binary with values {0, 255}. Your output consists of the following values:\n' + 
          str(np.unique(check_bin)))
else:
    print("Good, the output is binary with values {0, 255}.")

Again, we will make it easier for you to select the appropriate size and threshold values by adding an extra widget to the image display. Run the next cell to use the interactive widget.

In [ ]:
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the threshold slider
thresh_slider = widgets.IntSlider(value=125, min=0, max=255, step=1, description='threshold')
# Instantiate the diconnect button
button = widgets.Button(description='Detect H-lines')

# Define the callback function of the button
def button_callback(image):
    # Run the disconnect function on the image with the size indicated by the slider
    output = detect_hlines(image, n=size_slider.value, threshold=thresh_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_scratch_display = viewer(test_scratch, title="Detect horizontal lines", new_widgets=[size_slider, thresh_slider, button], 
                              callbacks=[button_callback], widgets=True)

In the following cell, assign `n` with the size of the structuring element and `threshold` with the threshold you think works best for this task.

In [ ]:
# Assign your values here
n = None
threshold = None

# YOUR CODE HERE

In [ ]:
# Perform a sanity check on n
if not 0 < n < 50: 
    print('The chosen size doesn\'t really make sense.')

In [ ]:
# Perform a sanity check on threshold
if not 0 < threshold < 255: 
    print('The chosen threshold does not really make sense.')

## 1.C. Combining morphological filters
[Back to table of contents](#ToC_2_Morphology)

In this section, you can use **one or several** of the morphological operators followed by a threshold operation. **For 1 point**, detect **the 23 round white objects of diameter $20 \pm 4$ pixels (the smaller round objects)** in the image `test_scratch`. Try to preserve their original shapes. Select an appropriate threshold value to get a binary image.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="test-scratch round showcase" src="images/test-scratch_round_showcase.png" width="500"><br>
      <em style="color: grey">Using morphological operations, you can get the image on the right from the image on the left.</em>
  </p>
</td>
</tr></table>

Insert your code into the function `detect_round(img)`.

<div class="alert alert-success">
<b>Hints:</b> <ul><li>Because you can use as many operators as you like with different structuring elements of multiple sizes, it would be too complicated to generate interactive sliders for all of them. That means you need to <b>hard-code the sizes of the structuring elements and the threshold value directly into the code</b> and change them by hand to find the a combination that works for you.</li><li>Design your workflow thinking in terms of simple tasks! For example, what kind of features do you want to keep or get rid of, and what size are they? The effect of each of the operators can be summerized in a effect, e.g. getting rid of big objects.</li></ul>
</div>

In [ ]:
# Function that detects roundish white objects of diamater 20 +/- 4 pixels
def detect_round(img):
    # Initialize the output image
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return output

# Run the function on the image test_scratch
test_scratch_modified = detect_round(test_scratch)

# Display the images
plt.close('all')
images = [test_scratch, test_scratch_modified]
titles = ['Original', 'Round objects detected']

test_round_viewer = viewer(images, title=titles, subplots=(1,2))

Run the next cell to check that your image is binary.

In [ ]:
# Check if the image consists of only one value
if len(np.unique(detect_round(test_scratch))) == 1:
       print(f"WARNING!\nYour image consists of one value: {np.unique(detect_round(test_scratch))}. You probably made a mistake in your code.\n")
# Check if the image is binary
if len(np.unique(detect_round(test_scratch))) != 2:
       print(f"WARNING!\nYour image is not binary, it still consists of {len(np.unique(detect_round(test_scratch)))} different values. Check your thresholding operation.")
else:
    print(f'Good, the image is binary.')

In [ ]:
# Check that the lower binary value is 0
if np.unique(detect_round(test_scratch))[0] != 0: 
    print(f"The lower binary value should be 0, not {np.unique(detect_round(test_scratch))[0]}.")
else:
    print(f'The lower binary value is correct.')

In [ ]:
# Check that the upper binary value is 255
if np.unique(detect_round(test_scratch))[1] != 255: 
    print(f"The upper binary value should be 255, not {np.unique(detect_round(test_scratch))[1]}.")
else:
    # Print victory message
    print(f'The upper binary value is correct.')

# 2. Lantuéjoul's skeleton (1 point)
[Back to table of contents](#ToC_2_Morphology)

In this section we're going to implement a 2D skeletonizing algorithm, Lantuéjoul's algorithm. This process is commonly used in handwritten text recognition, fingerprint validation and [raster-to-vector](https://en.wikipedia.org/wiki/Image_tracing) conversion. It is an iterative erosion procedure that gives an approximation of the skeleton of an object. The input is a binary image (object = 255, background = 0). The output is also a binary image (skeleton = 255, background = 0).

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="skeletonize showcase" src="images/skeletonize_showcase.png" width="500"><br>
<em style="color: grey">Lantuéjoul's algorithm: original image (left) and simple Lantuéjoul's algorithm (right).</em> 
  </p>
</td>
</tr></table>

The algorithm can be summarized as follows:
<hr>
while the image is not completely eroded:

<ol>
    <li>start from $n=1$</li>
    <li>apply erosion with a $3\times3$ cross element to get an eroded image $e_n$</li>
    <li>apply tophat with a $3\times3$ square element to $e_n$ to get the $n^{th}$ skeleton $s_n$</li>
    <li>take the union of $s_n$ with previous ones: $\mathrm{skel} =s_n \cup (s_{n-1} \cup s_{n-2} \cup \ldots \cup s_1)$</li>  
    <li>$n=n+1$</li>
</ol>
print(n)
<hr>

## 2.A. Implementing skeletonize
[Back to table of contents](#ToC_2_Morphology)

**For 1 point**, complete the function `skeletonize` that implements the above process and that writes the counter $n$ in the console to indicate **in total** how many skeletons have been used. Test your code on the image `test_skeleton`.

<div class="alert alert-info">

<b>Hints</b>: <ul><li>You can use <a href="https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html"><code>np.count_nonzero(img)</code></a> to get the number of non-zero pixels in <code>img</code>.</li>

<li>The correct way to get the union of two images is to use <a href="https://numpy.org/doc/stable/reference/generated/numpy.bitwise_or.html"><code>np.bitwise_or(img_1, img_2)</code></a></li></ul>
</div>

<div class="alert alert-warning">
<b>Beware:</b> If you don't set the correct stopping condition in the while loop, it can run forever and block the execution of all other code. The cell should generate its output in a few seconds, otherwise it is likely that you created an infinite loop. If this happens you can click on <code>Kernel</code> in the toolbar on top and select <code>Interrupt</code> to stop the infinite loop. After that you can adjust your code and rerun the cell.

</div>

In [ ]:
# Function that takes as input a binary image and returns its skeleton
def skeletonize(img):
    # Defining the output image (an array of zeros with the same shape as the input image of type 'uint8')
    output = np.zeros(img.shape, np.uint8)
    
    # YOUR CODE HERE
    return output
    
# Run the function on the test image
test_skeleton = skeletonize(test_img)
    
# Define the lists of images and names
images = [test_img, test_skeleton]
titles = ['Original', 'Skeleton']

# Display the images
plt.close('all')
skeletonize_viewer = viewer(images, title=titles, subplots=(1,2))

Run the cell below to verify that the output of your function is binary.

In [ ]:
# Check that the output is binary
if (len(np.unique(test_skeleton)) != 2 or 
    np.max(np.unique(test_skeleton)) != 255 or 
    np.min(np.unique(test_skeleton)) != 0): 
    print('WARNING!\nThe output is not binary with values {0, 255}.')
else:
    print('Good, the output is binary.')

## 2.B. Testing skeletonize
[Back to table of contents](#ToC_2_Morphology)

Apply your function `skeletonize` on the `christmas` image and display the result using the `viewer` by running the next cell.

<div class='alert alert-info'>
<b>Hint</b>: If you implemented <code>skeletonize</code> correctly, you should see <b>exactly</b> $1700$ white pixels.
</div>

In [ ]:
# YOUR CODE HERE

In the next cell, assign to the variable `N` the number of iterations it took to generate the skeleton of the `christmas` image.

In [ ]:
# Number of erosions to skeletonize the image
N = None
# YOUR CODE HERE

In [ ]:
# Perform sanity check on n
if not 0 < N < 200: 
    print('WARNING!\nThe selected number of erosions is not really reasonable.')

# 3. Distance map (1 point)
[Back to table of contents](#ToC_2_Morphology)

In this exercise we want to create an algorithm that can determine the distance of each pixel of an object to its nearest border. The input to this algorithm will be a binary image (object = $255$, background = $0$), like in the last exercise, and the output will be a distance map `M`, where the value of each pixel corresponds to the distance of this pixel to the background. 

In other words, the distance of a pixel is defined as the number of iterations needed to erode the image until the value of this pixel becomes $0$.
This means that background pixels (value = 0) will always have a distance of 0; the outermost pixels of an object will have distance 1; the second-outermost pixels a distance of 2 and so on. 

In the next cell, **for 1 point**, complete the function `distance_border_map`, that takes as parameter a binary image `img`, and returns the distance map `M` of the same size as `img`.
This function erodes the input image with a **disc structuring element of size $3\times3$** and simultaneously builds up the distance map `M` until there are only background pixels (value = 0) remaining. An example of the expected distance map is given below:

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="distance_from_center_showcase" src="images/distance_from_center_showcase.png" width="400"><br>
<em style="color: grey">Tracking the erosions it takes to erase a pixel, you can build a distance map.</em>
  </p>
</td>
</tr></table>



In [ ]:
def distance_from_border(img):
    # Check that the input image is binary
    if not (np.unique(img) == np.array([0, 255])).all(): 
        raise ValueError('img should only contain the values {0, 255}.')
        
    # Distance map
    M = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return M

Now run the next cell. It will apply your function on the image `hela` and test that the result has continuous values in the range $[0, max]$, where $max$ is the highest value in the result. Moreover, it will visualize the result.

<div class='alert alert-info'>
<b>Notes</b>: <ul>
    <li>You can change the colormap of the viewer in the <code>Options</code> menu for better visualization of the distance map!</li>
    <li>Reflect on the highest value on the image! For example, it for sure should not be larger than the largest dimension in the image.</li>
    </ul>
</div>

In [ ]:
# First we apply distance_from_border to hela
hela_dist_map = distance_from_border(hela)

if not np.array_equal(np.unique(hela_dist_map), np.arange(int(np.max(hela_dist_map)) + 1)):
    print(f'WARNING!!!\nYour function does not have uniform values between 0 and the maximum of the distance map ({np.max(hela_dist_map)})')
else:
    print('Good job! The result of your function seems to make sense.')

plt.close('all')
viewer([hela, hela_dist_map], title=['Original', 'Distance map'], subplots=(1,2))


# 4. Cartoonize your picture!
[Back to table of contents](#ToC_2_Morphology)

Choose a natural picture of your choice from the internet or from your own collection (it shouldn't be too large, otherwise the operations will take a long time) or simply the RGB _natural_image.jpg_ we provide here. 

<div class='alert alert-info'>
    Remember that the morphological operators from skimage only work on binary or grayscale images. However, we can treat each channel of the RGB image as a grayscale image, perform the same operations on each them, and combine the three channels again to get the output image! In the cell below, you will find an example how to do this.
    </div>

In [ ]:
# YOUR CODE HERE

Now it's time to build your own workflow! In the next cell, be creative in the combination of morphological operators, arithmetic operators, and threshold operations to give a cartoon effect to your picture! You can also make it look like a painting or give it a distortion effect, explore the possibilities! Feel free to share your artwork on the Moodle forum of this course!

<div class="alert alert-success">
    Congratulations on finishing the Morphology lab!
</div>
    
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1123107">Moodle</a>, <b>in a zip file with the first part of this lab</b>.
</p>

* Keep the name of the notebook as *2_Morphology_Applications.ipynb*!
* Name the `zip` file *Morphology_lab.zip*.